# Phase 1: The Structured Pipeline
## 1. Business & Data Understanding
**Goal:** Prepare the "Environment" dataset. We will process the raw Airbnb calendar and enrich it with external weather data to create a robust structured dataset for occupancy prediction.

**Steps:**
1.  **Load & Clean:** Process `calendar.csv.gz` (Price cleaning, Target creation).
2.  **Enrich:** Merge with `weather_amsterdam.csv`.
3.  **Feature Engineering:** Create time-lagged features (Past Price) and seasonality markers.

In [2]:
import pandas as pd
import numpy as np
import sys
import os

# Add parent directory to path to import config
sys.path.append('..')
import config

print(f"✅ Setup Complete. Data Path: {config.DATA_RAW}")

✅ Setup Complete. Data Path: C:\Users\prajw\MSCDAD_B2025\Data Mining and Machine Learning\Amsterdam_Forecast\data\raw


## 2. Data Preparation (Cleaning & Imputation)
We load the Airbnb **Calendar** (Dataset A).

**Data Quality Fix:**
In the raw calendar data, the `price` column is often null (NaN) for dates that are already booked. To fix this, we:
1.  Load the **Listings** dataset (`listings.csv.gz`).
2.  Extract the `base_price` for every listing.
3.  **Merge** this base price into the calendar.
4.  **Impute:** If the daily price is missing, fill it with the listing's base price.

**Target Variable:**
* We map `available` column: `f` (False) $\to$ **1 (Occupied)**, `t` (True) $\to$ **0 (Available)**.ns.

In [3]:
# 1. Load Calendar
print("⏳ Loading Calendar Data...")
df_calendar = pd.read_csv(config.CALENDAR_FILE)

# 2. Load Listings (To rescue the missing prices)
print("⏳ Loading Listings Data (to fix missing prices)...")
df_listings = pd.read_csv(config.LISTINGS_FILE)

# Keep only ID and Price from listings
# The column in listings is usually 'id' and 'price'
df_listings_prices = df_listings[['id', 'price']].copy()
df_listings_prices.rename(columns={'id': 'listing_id', 'price': 'base_price'}, inplace=True)

# 3. Clean 'base_price' in listings
# It usually comes as "$150.00", so we clean it
if df_listings_prices['base_price'].dtype == 'O':
    df_listings_prices['base_price'] = df_listings_prices['base_price'].str.replace('$', '').str.replace(',', '').astype(float)

# 4. Merge Base Price into Calendar
print("🔄 Merging Base Prices into Calendar...")
df_calendar = pd.merge(df_calendar, df_listings_prices, on='listing_id', how='left')

# 5. Fix the Missing Price Column
# If calendar price is NaN, fill it with the base_price
if df_calendar['price'].dtype == 'O':
    df_calendar['price'] = df_calendar['price'].str.replace('$', '').str.replace(',', '').astype(float)

df_calendar['price'] = df_calendar['price'].fillna(df_calendar['base_price'])

# 6. Create Target & Final Cleanup
df_calendar['Occupancy_Status'] = df_calendar['available'].map({'f': 1, 't': 0})
df_calendar['date'] = pd.to_datetime(df_calendar['date'])

# Filter Columns (Now we use the filled 'price')
cols_to_keep = ['listing_id', 'date', 'price', 'minimum_nights', 'maximum_nights', 'Occupancy_Status']
df_calendar = df_calendar[cols_to_keep]

# DROP rows where price is STILL missing (e.g. if listing didn't exist in listings file)
df_calendar.dropna(subset=['price'], inplace=True)

print(f"✅ Calendar Loaded & Fixed. Shape: {df_calendar.shape}")
print(f"Missing Prices: {df_calendar['price'].isnull().sum()}") # Should be 0 now
df_calendar.head()

⏳ Loading Calendar Data...
⏳ Loading Listings Data (to fix missing prices)...
🔄 Merging Base Prices into Calendar...
✅ Calendar Loaded & Fixed. Shape: (2144010, 6)
Missing Prices: 0


,listing_id,date,price,minimum_nights,maximum_nights,Occupancy_Status
730,553514,2025-09-11,108.0,2,30,1
731,553514,2025-09-12,108.0,2,30,1
732,553514,2025-09-13,108.0,2,30,1
733,553514,2025-09-14,108.0,2,30,1
734,553514,2025-09-15,108.0,2,30,1


## 3. Weather Enrichment
We import the weather data generated by our script and merge it onto the calendar based on the `date`.

**Method:** Left Join.
**Imputation:** If any weather data is missing (NaN), we use 'Forward Fill' (ffill) assuming weather doesn't change drastically overnight.

In [4]:
# 1. Load Weather
weather_path = os.path.join(config.DATA_RAW, 'weather_amsterdam.csv')
df_weather = pd.read_csv(weather_path)

# FIX: Add dayfirst=True to handle "13-01-2023" correctly
df_weather['Date'] = pd.to_datetime(df_weather['Date'], dayfirst=True)

# 2. Merge
# Calendar is Left (Main), Weather is Right (Lookup)
df_merged = pd.merge(df_calendar, df_weather, left_on='date', right_on='Date', how='left')

# Drop the duplicate 'Date' column from weather
df_merged.drop(columns=['Date'], inplace=True)

# 3. Impute Missing Weather (if any)
# We fill forward first, then backward fill for any gaps at the very start
weather_cols = ['Temp', 'Rain', 'Wind']
df_merged[weather_cols] = df_merged[weather_cols].ffill().bfill()

print(f"✅ Weather Merged. New Shape: {df_merged.shape}")
print(f"Missing Values Check:\n{df_merged.isnull().sum()}")

✅ Weather Merged. New Shape: (2144010, 9)
Missing Values Check:
listing_id          0
date                0
price               0
minimum_nights      0
maximum_nights      0
Occupancy_Status    0
Temp                0
Rain                0
Wind                0
dtype: int64


## 4. Feature Engineering (Structured)
To help the model understand time and trends, we create:
1.  **Seasonality:** Extract Month and Day of Week.
2.  **Lag Features:** Calculate the price of the listing 7 days ago. This helps the model understand if a price drop led to a booking.

In [5]:
# 1. Seasonality
df_merged['month'] = df_merged['date'].dt.month
df_merged['day_of_week'] = df_merged['date'].dt.dayofweek # 0=Monday, 6=Sunday

# 2. Lag Features (Price 7 days ago)
# We must sort by listing and date to ensure shifts are correct
df_merged = df_merged.sort_values(['listing_id', 'date'])

# Group by listing and shift price by 7 rows (days)
df_merged['price_7d_lag'] = df_merged.groupby('listing_id')['price'].shift(7)

# Drop the first 7 days of data where lag is NaN
df_merged.dropna(subset=['price_7d_lag'], inplace=True)

print("✅ Feature Engineering Complete.")
df_merged[['listing_id', 'date', 'price', 'price_7d_lag']].head(10)

✅ Feature Engineering Complete.


,listing_id,date,price,price_7d_lag
1889247,27886,2025-09-18,132.0,132.0
1889248,27886,2025-09-19,132.0,132.0
1889249,27886,2025-09-20,132.0,132.0
1889250,27886,2025-09-21,132.0,132.0
1889251,27886,2025-09-22,132.0,132.0
1889252,27886,2025-09-23,132.0,132.0
1889253,27886,2025-09-24,132.0,132.0
1889254,27886,2025-09-25,132.0,132.0
1889255,27886,2025-09-26,132.0,132.0
1889256,27886,2025-09-27,132.0,132.0


## 5. Save Checkpoint
We save this processed "Structured" dataset. In Phase 3, we will merge this with the "Text" dataset.

In [6]:
# Save to processed folder
output_file = os.path.join(config.DATA_PROCESSED, '01_structured_data.csv')

# Note: We save index=False to keep it clean
df_merged.to_csv(output_file, index=False)

print(f"💾 Saved Structured Dataset to: {output_file}")

💾 Saved Structured Dataset to: C:\Users\prajw\MSCDAD_B2025\Data Mining and Machine Learning\Amsterdam_Forecast\data\processed\01_structured_data.csv
